In [1]:
import sqlite3

# Establishing our Database path.  If no database exists here, this will create one.
sqlite_db = 'data/sql/test_db.sqlite'

# Establishing the SQL Connection to our Database.  
conn = sqlite3.connect(sqlite_db)

c = conn.cursor()
# Cursor objects allow you to keep track of which result set is which, 
# since it's possible to run multiple queries before you're done fetching the results of the first.
# CURSORS seem to be a tool for iterating over tables row-by-row
# This is a conceptual SQL object that is hard to give a very clear consise definition of.

In [3]:
# We can use the 'execute' fucntion on our Database Connection (With Cursor) to execute a SQL Query.
c.execute('CREATE TABLE houses1 (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')

# sqlite.connect('data/sql/test_db.sqlite').cursor().execute()

# Save (commit) the changes (Just like GitHub)
conn.commit()

In [4]:
# We can create a tuple
last_sale = (None, 4000, 5, 22, 619000)

# We can insert said tuple into the database table using character substitution.
c.execute('INSERT INTO houses1 VALUES (?, ?, ?, ?, ?)',last_sale)

# Remember to commit the changes
conn.commit()

In [6]:
# We can use the same logic to insert a whole list of tuples!
# Tell me thats not helpful!
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

c.executemany('INSERT INTO houses1 VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

In [9]:
import pandas as pd
from pandas.io import sql

cars = pd.read_csv('data/csv/car-names.csv', encoding = 'utf-8')

# If you don't specify the type encoding as 'utf-8' you're going to have a bad time when you try to convert to SQL

In [10]:
# Checking what our dataframe looks like
cars.head(3)

,Id,Model,Make
0,1,'chevrolet','chevrolet chevelle malibu'
1,2,'buick','buick skylark 320'
2,3,'plymouth','plymouth satellite'


In [11]:
# Checking for nulls in our data
cars.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 3 columns):
Id       406 non-null int64
Model    406 non-null object
Make     406 non-null object
dtypes: int64(1), object(2)
memory usage: 9.6+ KB


In [12]:
# Import Sqlite3 Library
import sqlite3

# Establishing the Connection to our Database.  If no database exists here, this will create one.
connection = sqlite3.connect('data/sql/Cars.db.sqlite')

# Keep in mind the directory your notebook is open in is the base directory for all of our SQL actions from here.

In [13]:
# Converts a DataFrame into a SQL database
cars.to_sql(name = 'car_names', con = connection, if_exists = 'replace', index = False)

# name = name of the database useful if you have multiple tables in a SQL database
# con = the connection path to where the data should be placed
# if_exists = condition to pass if the database already exists.

In [14]:
# Creating a Table for Order Breakdowns
makers = pd.read_csv('data/csv/car-makers.csv', encoding = 'utf-8')

connection = sqlite3.connect('data/sql/Cars.db.sqlite')

makers.to_sql(name = 'car_makers', con = connection, if_exists = 'replace', index = False)

In [15]:
# Creating a Table for the Sales Targets
data = pd.read_csv('data/csv/cars-data.csv',encoding = 'utf-8')

connection = sqlite3.connect('data/sql/Cars.db.sqlite')

data.to_sql(name = 'car_data', con = connection, if_exists = 'replace', index = False)

In [16]:
#The SQL Sub-library from Pandas will allow us to run SQL queries within python.
from pandas.io import sql
# We already imported sqlite3, but it will also be needed for reading in SQL 
import sqlite3

# Specifying the SQL Path to the SQL Database
connection = sqlite3.connect('data/sql/Cars.db.sqlite')

# This is our SQL Query
query = 'select * from car_names'

# Using the read_sql from the Pandas SQL library and setting it equal to a DF object.
cars = sql.read_sql(query, con = connection)

cars.head()

,Id,Model,Make
0,1,'chevrolet','chevrolet chevelle malibu'
1,2,'buick','buick skylark 320'
2,3,'plymouth','plymouth satellite'
3,4,'amc','amc rebel sst'
4,5,'ford','ford torino'


In [17]:
sql.read_sql('select * from car_names', connection).head()

#If we 'shift+tab' we can see that the read_sql function takes the arguments 'sql' and 'con'

#sql = Our SQL query as a string
#con = is the connection path to our SQL Database

,Id,Model,Make
0,1,'chevrolet','chevrolet chevelle malibu'
1,2,'buick','buick skylark 320'
2,3,'plymouth','plymouth satellite'
3,4,'amc','amc rebel sst'
4,5,'ford','ford torino'


In [18]:
# In the case that typing out sql.read_sql() is a little too much,
# we'll create a function shortcut.


CARS = sqlite3.connect('data/sql/Cars.db.sqlite')


def Q(query, db=CARS):
    return sql.read_sql(query, db)

In [19]:

Q('select * from car_names limit 5')

,Id,Model,Make
0,1,'chevrolet','chevrolet chevelle malibu'
1,2,'buick','buick skylark 320'
2,3,'plymouth','plymouth satellite'
3,4,'amc','amc rebel sst'
4,5,'ford','ford torino'


In [20]:
new_car = (None, 'Ferrari','The Ferrari')
CARS.execute('INSERT INTO car_names VALUES (?, ?, ?)',new_car)

In [21]:
new_car = [None, 'Testla', None]
CARS.execute('INSERT INTO car_names VALUES (?, ?, ?)',new_car)

In [22]:
Q('SELECT * FROM car_names WHERE car_names."Model" = "Testla"')

,Id,Model,Make
0,None,Testla,None


In [23]:
Q('select * from car_makers limit 5')

,Id,Maker,FullName,Country
0,1,'amc','American Motor Company',1
1,2,'volkswagen','Volkswagen',2
2,3,'bmw','BMW',2
3,4,'gm','General Motors',1
4,5,'ford','Ford Motor Company',1


In [24]:
Q('select * from car_data limit 5')

,Id,MPG,Cylinders,Edispl,Horsepower,Weight,Accelerate,Year
0,1,18,8,307.0,130,3504,12.0,1970
1,2,15,8,350.0,165,3693,11.5,1970
2,3,18,8,318.0,150,3436,11.0,1970
3,4,16,8,304.0,150,3433,12.0,1970
4,5,17,8,302.0,140,3449,10.5,1970


In [25]:
inner_join = Q('SELECT car_names."Make", car_data."MPG", car_data."Horsepower", car_data."Year" '
'FROM car_names '
'INNER JOIN car_data '
'ON car_names."Id"=car_data."Id"')
inner_join.head()

,Make,MPG,Horsepower,Year
0,'chevrolet chevelle malibu',18,130,1970
1,'buick skylark 320',15,165,1970
2,'plymouth satellite',18,150,1970
3,'amc rebel sst',16,150,1970
4,'ford torino',17,140,1970


In [26]:
inner_join.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 4 columns):
Make          406 non-null object
MPG           406 non-null object
Horsepower    406 non-null object
Year          406 non-null int64
dtypes: int64(1), object(3)
memory usage: 12.8+ KB


In [27]:
left_join = Q('SELECT car_names."Make", car_data."MPG", car_data."Horsepower", car_data."Year" '
'FROM car_names '
'LEFT JOIN car_data '
'ON car_names."Id"=car_data."Id"')
left_join.head()

,Make,MPG,Horsepower,Year
0,'chevrolet chevelle malibu',18,130,1970.0
1,'buick skylark 320',15,165,1970.0
2,'plymouth satellite',18,150,1970.0
3,'amc rebel sst',16,150,1970.0
4,'ford torino',17,140,1970.0


In [28]:
left_join.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 4 columns):
Make          407 non-null object
MPG           406 non-null object
Horsepower    406 non-null object
Year          406 non-null float64
dtypes: float64(1), object(3)
memory usage: 12.8+ KB
